## Data Inspiration
<p>I was watching cricket match yesterday. India vs WestIndies. Both Rohit and Virat were well settled on crease with 200 run partnership and counting. virat was close to 150 and i saw a dialog box on TV stating kohli just needs another 154 runs to reach 10k Runs. To join the groups of elite Batsmen including Sachin, Pointing and Dravid. I was little surprised to see him reach close to the milestone so soon. </p>
<p>Then it stuck me, the amount of data in cricket. There's huge data in cricket. Data from 4's and 6's by a batsman in a match to number of balls faced in his test career.</p>
<p>It din't take me long to search for perfect website to find those stats. In India, cricket is a religion after all. I landed on cricinfo, searched for stats and found <a href='http://stats.espncricinfo.com/ci/content/records/335431.html'>this</a> webpage containing data of every single test match ever recorded or reported.</p>
<p>So, i dug in, scraped the data to analyse every stat i could imagine</p>

In [1]:
import requests

In [2]:
r = requests.get('http://stats.espncricinfo.com/ci/content/records/335431.html')
r.text[0:500]

'\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<!-- hostname: web005, edition-view: espncricinfo-en-in, country: unknown, cluster: www, created: 2018-10-26 04:42:10 -->\n<html xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="http://developers.facebook.com/schema/" >\n<head>\n <script type="text/javascript">var _sf_startpt=(new Date())'

In [3]:
from bs4 import BeautifulSoup  
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
stats = soup.find_all('tr', attrs={'class':'data1'})
print("The resulting object is of type {}".format(type(stats)))
print("Total number of results, i.e total no of tests played till date are: {}".format(len(stats)))

The resulting object is of type <class 'bs4.element.ResultSet'>
Total number of results, i.e total no of tests played till date are: 738


The Whole Data is present in form of a list in the <i>bs4.element.ResultSet</i>. 
<p>Each test series is in form of an element in list.</p>
<p>Below is the structure of data in each element</p>

In [5]:
stats[0]

<tr class="data1" data-days="685654">
<td class="left"><a class="data-link" href="/ci/engine/series/60260.html">England in Australia Test Series</a></td>
<td class="left" nowrap="nowrap">1876/77</td>
<td nowrap="nowrap"></td>
<td class="left" nowrap="nowrap"><a class="data-link" href="/ci/content/team/427988.html">drawn</a></td>
<td nowrap="nowrap">1-1 (2)</td>
</tr>

### Data Structure of each table row of the data table
- The Number 685654 in data-days attribute. Such numbers notably unique are present in all elements. must mean something.
- The pathname of link to direct to page containing the test series results
- The Test Series teams playing. It gives information about the home team and the away team.
- The year the test series is played in
- A link inside 4th td tag leading to the winning team's page. It's throwing 404 (Page not Found) error for drawn series.
- The numerical Result of the series
- Total number of matches played in the series. ( Given in the Bracket )

So, the features (and it's values as per the first row) can be :
<p>
    <b>data-days</b> 685654<br>
    <b>Match link</b> /ci/engine/series/60260.html<br>
    <b>teams playing</b> England in Australia Test Series<br>
    <b>Year</b> 1876/77<br>
    <b>Team Won</b> Drawn<br>
    <b>Result</b> 1-1 (first number corresponds to winner)<br>
    <b>Matches Played</b> 2<br>
</p>

<p>The Dataframe would look something like this: </p>
<table>
    <thead>
    <tr>
        <td><b>data-days</b></td>
        <td><b>Series link</b></td>
        <td><b>Teams playing</b></td>
        <td><b>Year</b></td>
        <td><b>Team Won</b></td>
        <td><b>Result</b></td>
        <td><b>Matches Played</b></td>
    </tr>
    </thead>
    <tr>
        <td>685654<br></td>
        <td>/ci/engine/series/60260.html<br></td>
        <td>England in Australia Test Series<br></td>
        <td>1876/77<br></td>
        <td>Drawn<br></td>
        <td>1-1<br></td>
        <td>2<br></td>
    </tr>
</table>

- We can Further divide the Team playing into Home team and away team. Some of the Series have Series name embedded in this section. So, we can add another column <b>Series Name</b> to it. 
- We can divide the Year to series <b>starting Year</b> and <b>ending Year</b>
- The Result can be divided into <b>Home wins</b> and <b>away wins</b>

<p>With updated features, the dataframe would look like this: </p>
<table>
    <thead>
    <tr>
        <td><b>datadaysId</b></td>
        <td><b>Link</b></td>
        <td><b>Name</b></td>
        <td><b>Home</b></td>
        <td><b>Away</b></td>
        <td><b>startYear</b></td>
        <td><b>endYear</b></td>
        <td><b>TeamWon</b></td>
        <td><b>HomeWins</b></td>
        <td><b>AwayWins</b></td>
        <td><b>Matches</b></td>
    </tr>
    </thead>
    <tr>
        <td>685654</td>
        <td>/ci/engine/series/60260.html</td>
        <td></td>
        <td>Australia</td>
        <td>England</td>
        <td>1876</td>
        <td>1877</td>
        <td>Drawn</td>
        <td>1</td>
        <td>1</td>
        <td>2</td>
    </tr>
</table>

Now that we have format of the features, let's separate the values from the bs4 Resultset

In [6]:
final_result = stats[-2]
final_result

<tr class="data1" data-days="737346">
<td class="left"><a class="data-link" href="/ci/engine/series/1157748.html">West Indies in India Test Series</a></td>
<td class="left" nowrap="nowrap">2018/19</td>
<td nowrap="nowrap"></td>
<td class="left" nowrap="nowrap"><a class="data-link" href="/ci/content/team/6.html">India</a></td>
<td nowrap="nowrap">2-0 (2)</td>
</tr>

This series is the inspiration for my data. So, this data is updated till date.

In [7]:
records = []
for series in stats:
    datadaysId = int(series['data-days'])
    Link = series.find('a').get('href')
    teams_text = series.find_all('td')[0].text
    years = series.find_all('td')[1].text
    winner = series.find_all('td')[3].text
    result = series.find_all('td')[4].text
    records.append((datadaysId, Link, teams_text, years, winner, result))

In [8]:
records[-3:]

[(737313,
  '/ci/engine/series/1119531.html',
  'Pataudi Trophy (India in England)',
  '2018',
  'England',
  '4-1 (5)'),
 (737346,
  '/ci/engine/series/1157748.html',
  'West Indies in India Test Series',
  '2018/19',
  'India',
  '2-0 (2)'),
 (737351,
  '/ci/engine/series/1157363.html',
  'Pakistan v Australia Test Series (in United Arab Emirates)',
  '2018/19',
  'Pakistan',
  '1-0 (2)')]

In [9]:
import pandas as pd
series = pd.DataFrame(records, columns=['datadaysID', 'link', 'teams_text', 'years', 'winner', 'result'])

In [10]:
series.head(5)

,datadaysID,link,teams_text,years,winner,result
0,685654,/ci/engine/series/60260.html,England in Australia Test Series,1876/77,drawn,1-1 (2)
1,686294,/ci/engine/series/60261.html,England in Australia Test Match,1878/79,Australia,1-0 (1)
2,686907,/ci/engine/series/60262.html,Australia in England Test Match,1880,England,1-0 (1)
3,687459,/ci/engine/series/60263.html,England in Australia Test Series,1881/82,Australia,2-0 (4)
4,687627,/ci/engine/series/60264.html,Australia in England Test Match,1882,Australia,1-0 (1)


In [11]:
series.tail(6)

,datadaysID,link,teams_text,years,winner,result
732,737236,/ci/engine/series/1135145.html,Sobers/Tissera Trophy (Sri Lanka in West Indies),2018,drawn,1-1 (3)
733,737254,/ci/engine/series/1146713.html,Bangladesh in West Indies Test Series,2018,West Indies,2-0 (2)
734,737263,/ci/engine/series/1142576.html,South Africa in Sri Lanka Test Series,2018,Sri Lanka,2-0 (2)
735,737313,/ci/engine/series/1119531.html,Pataudi Trophy (India in England),2018,England,4-1 (5)
736,737346,/ci/engine/series/1157748.html,West Indies in India Test Series,2018/19,India,2-0 (2)
737,737351,/ci/engine/series/1157363.html,Pakistan v Australia Test Series (in United Ar...,2018/19,Pakistan,1-0 (2)


## Feature Value distribution

Let's now see how the data is distributed across the whole Dataframe and if any data is outside the patern we assumed for each feature.

### 1. datadaysID

In [12]:
print("Total number of datadaysID elements are {}, number of unique id elements are {}".format(len(series), series['datadaysID'].nunique()))

Total number of datadaysID elements are 738, number of unique id elements are 729


9 datadaysID values are repeated. Lets check some of the most frequent datadaysID values

In [13]:
# Top 3 most occuring datadaysID values
series['datadaysID'].value_counts()[:3]

731538    2
733630    2
732576    2
Name: datadaysID, dtype: int64

We can see that some of the datadaysID are not unique. There aren't any specific patterns for similar datadaysID values too (as shown below). Since we don't even know what the field represents in cricketing terminology, we can ignore the feature while analysing. But let's save it for now in the Dataframe.

In [14]:
series[series['datadaysID'] == 731538]

,datadaysID,link,teams_text,years,winner,result
475,731538,/ci/engine/series/60737.html,Pakistan in Zimbabwe Test Series,2002/03,Pakistan,2-0 (2)
476,731538,/ci/engine/series/60736.html,Sri Lanka in South Africa Test Series,2002/03,South Africa,2-0 (2)


### 2. link

In [15]:
print("Total number of links are {}, number of unique links are {}".format(len(series), series['link'].nunique()))

Total number of links are 738, number of unique links are 738


Each link representing a html page containing scorecard summary of each match in that series. There are no duplicate links.

### 3. teams_text

In [16]:
series['teams_text'].value_counts()[:5]

The Ashes (Australia in England)              35
The Ashes (England in Australia)              35
England in New Zealand Test Series            18
New Zealand in England Test Series            17
The Wisden Trophy (West Indies in England)    16
Name: teams_text, dtype: int64

The most common series is "The Ashes" with most number of matches played. 
<p>Some of the Series have names. All the teams we checked are in (Team A in Team B) format. <br>
    We can find the countries by slicing at <b style="color: green"><u>in</u></b> and checking if the strings (countries or part of their country names) are present in test playing nations. </p>
<p>We can now check if all the text fields have exactly 2 names of test playing nations once we break the strings</p>

In [17]:
test_playing_nations = ['India', 'England', 'Australia', 'South Africa', 'West Indies', 'New Zealand', 'Pakistan', 'Sri Lanka', 'Zimbabwe', 'Bangladesh', 'Ireland', 'Afghanistan']

In [18]:
import re
import numpy as np
def split_teams(team, separator):
    team = re.sub(r'[^a-zA-Z]', ' ', team).split()
    result = []
    for idx, name in enumerate(team):
        if name.lower() == separator:
            # team 1 (home)
            if team[idx-1] in test_playing_nations:
                result.append(team[idx-1])
            elif (team[idx-2] + ' ' + team[idx-1]) in test_playing_nations:
                result.append(team[idx-2] + ' ' + team[idx-1])
            else:
                result.append(np.NaN)
            # team 2 (away)
            if team[idx+1] in test_playing_nations:
                result.append(team[idx+1])
            elif (team[idx+1] + ' ' + team[idx+2]) in test_playing_nations:
                result.append(team[idx+1] + ' ' + team[idx+2])
            else:
                result.append(np.NaN)
            return result
        
#     return set(test_playing_nations) & set(team)

# # np.char.array(q) + ' ' + np.char.array(q)

In [19]:
series['Home'] = series['teams_text'].apply(lambda x: split_teams(x, 'in')[1])
series['Away'] = series['teams_text'].apply(lambda x: split_teams(x, 'in')[0])

In [20]:
series.head()

,datadaysID,link,teams_text,years,winner,result,Home,Away
0,685654,/ci/engine/series/60260.html,England in Australia Test Series,1876/77,drawn,1-1 (2),Australia,England
1,686294,/ci/engine/series/60261.html,England in Australia Test Match,1878/79,Australia,1-0 (1),Australia,England
2,686907,/ci/engine/series/60262.html,Australia in England Test Match,1880,England,1-0 (1),England,Australia
3,687459,/ci/engine/series/60263.html,England in Australia Test Series,1881/82,Australia,2-0 (4),Australia,England
4,687627,/ci/engine/series/60264.html,Australia in England Test Match,1882,Australia,1-0 (1),England,Australia


Looks good, We separated out both home and away teams. let's see if any null values were created while breaking the teams string.

In [21]:
series[series['Home'].isnull()]

,datadaysID,link,teams_text,years,winner,result,Home,Away
461,731256,/ci/engine/series/60722.html,Pakistan v West Indies Test Series (in United ...,2001/02,Pakistan,2-0 (2),NaN,NaN
606,734465,/ci/engine/series/461555.html,Pakistan v South Africa Test Series (in United...,2010/11,drawn,0-0 (2),NaN,NaN
620,734813,/ci/engine/series/530421.html,Pakistan v Sri Lanka Test Series (in United Ar...,2011/12,Pakistan,1-0 (3),NaN,NaN
628,734904,/ci/engine/series/531617.html,Pakistan v England Test Series (in United Arab...,2011/12,Pakistan,3-0 (3),NaN,NaN
653,735532,/ci/engine/series/649077.html,Pakistan v South Africa Test Series (in United...,2013/14,drawn,1-1 (2),NaN,NaN
658,735618,/ci/engine/series/657619.html,Pakistan v Sri Lanka Test Series (in United Ar...,2013/14,drawn,1-1 (3),NaN,NaN
669,735905,/ci/engine/series/727915.html,Pakistan v Australia Test Series (in United Ar...,2014/15,Pakistan,2-0 (2),NaN,NaN
671,735932,/ci/engine/series/742603.html,Pakistan v New Zealand Test Series (in United ...,2014/15,drawn,1-1 (3),NaN,NaN
685,736272,/ci/engine/series/902621.html,Pakistan v England Test Series (in United Arab...,2015/16,Pakistan,2-0 (3),NaN,NaN
700,736636,/ci/engine/series/1050207.html,Pakistan v West Indies Test Series (in United ...,2016/17,Pakistan,2-1 (3),NaN,NaN


<p>Quite a few. These strings are in <b>C1 v C2(in C3)</b> format. </p>
<p>All these matches have pakisthan in common. Terrorists attacked Sri lankan bus carrying their cricketers to the stadium to play pakisthan in 2011(i guess). Few were injured. So, cricketing nations banned playing in pakisthan citing it unsafe. Pakisthan had to move their venue to a new place, United Arab Emirates, their new home for the matches to come. </p>
<p>We have to break the countries with <b>v</b> rather than in to get the Home Away teams</p>

In [22]:
series[series['Away'].isnull() & series['Home'].notnull()]

,datadaysID,link,teams_text,years,winner,result,Home,Away
36,698562,/ci/engine/series/60297.html,Australia v South Africa Test Series (in England),1912,Australia,2-0 (3),England,NaN
418,730437,/ci/engine/series/60678.html,South Africa v Zimbabwe Test Series (in South ...,1999/00,South Africa,2-0 (2),South Africa,NaN
472,731510,/ci/engine/series/60732.html,Australia v Pakistan Test Series (in Sri Lanka...,2002/03,Australia,3-0 (3),Sri Lanka,NaN
530,732601,/ci/engine/series/209960.html,ICC Super Series Test Match (ICC World XI in A...,2005/06,Australia,1-0 (1),Australia,NaN


3 more neutral venues with C1 v C2 ( in C3 ) format. A Super Series Match between then World Champions and Rest of the world.

In [23]:
series.loc[530, 'Away'] = 'ICC World XI'
series.loc[series['Away'].isnull() & series['Home'].notnull(), 'Home'] = np.NaN

In [24]:
series.loc[series['Home'].isnull(), 'Home'] = series['teams_text'][series['Home'].isnull()].apply(lambda x: split_teams(x, 'v')[0])
series.loc[series['Away'].isnull(), 'Away'] = series['teams_text'][series['Away'].isnull()].apply(lambda x: split_teams(x, 'v')[1])

In [25]:
series.tail()

,datadaysID,link,teams_text,years,winner,result,Home,Away
733,737254,/ci/engine/series/1146713.html,Bangladesh in West Indies Test Series,2018,West Indies,2-0 (2),West Indies,Bangladesh
734,737263,/ci/engine/series/1142576.html,South Africa in Sri Lanka Test Series,2018,Sri Lanka,2-0 (2),Sri Lanka,South Africa
735,737313,/ci/engine/series/1119531.html,Pataudi Trophy (India in England),2018,England,4-1 (5),England,India
736,737346,/ci/engine/series/1157748.html,West Indies in India Test Series,2018/19,India,2-0 (2),India,West Indies
737,737351,/ci/engine/series/1157363.html,Pakistan v Australia Test Series (in United Ar...,2018/19,Pakistan,1-0 (2),Pakistan,Australia


### Home Away Format from teams text

<table align="left" style="width: 50%;">
    <tr>
        <td></td>
        <th>T1 in T2</th>
        <th>T1 v T2</th>
    </tr>
    <tr>
        <th>Home</th>
        <td>T2</td>
        <td>T1</td>
    </tr>
    <tr>
        <th>Away</th>
        <td>T1</td>
        <td>T2</td>
    </tr>
</table>

In [26]:
# no missing values
series[['Home', 'Away']].isnull().sum()

Home    0
Away    0
dtype: int64

### 4. years

season can be in one year or across 2 years if it starts at the end of the year.

In [27]:
# divide year to starting and ending year
def years(year):
    if '/' in year:
        year = year.split('/')
        year[1] = year[0][:2] + year[1]
        return tuple(map(int, year))
    else:
        return int(year), int(year)

In [28]:
series = series.merge(series['years'].apply(lambda x: pd.Series({'start': years(x)[0], 'end': years(x)[1]})), left_index=True, right_index=True)
series.head()

,datadaysID,link,teams_text,years,winner,result,Home,Away,start,end
0,685654,/ci/engine/series/60260.html,England in Australia Test Series,1876/77,drawn,1-1 (2),Australia,England,1876,1877
1,686294,/ci/engine/series/60261.html,England in Australia Test Match,1878/79,Australia,1-0 (1),Australia,England,1878,1879
2,686907,/ci/engine/series/60262.html,Australia in England Test Match,1880,England,1-0 (1),England,Australia,1880,1880
3,687459,/ci/engine/series/60263.html,England in Australia Test Series,1881/82,Australia,2-0 (4),Australia,England,1881,1882
4,687627,/ci/engine/series/60264.html,Australia in England Test Match,1882,Australia,1-0 (1),England,Australia,1882,1882


### 5. result

<b>Format</b>: matches winner won - matches loser won ( total matches won )

In [29]:
print("empty values in result: {}".format(sum(series['result'] == '')))
missing_result = series[series['result'] == ''].index.tolist()
print(missing_result)

empty values in result: 8
[39, 206, 225, 229, 288, 357, 411, 463]


In [30]:
# drop corresponsing rows
series.drop(missing_result, inplace=True)

In [31]:
# Function to break the result into [home wins, away wins, total matches, total draw]
def result_break(row):
#     print(row)
    res = row['result'].split()
    result = res[0].split('-')
    if row['winner'] == row['Away']:
        result.reverse()
    result.append(re.sub(r'[^\d+]', '', res[1]))
    result = list(map(int, result))
    result.append(result[2] - (result[0]+result[1]))
    return pd.Series({'home_wins': result[0], 'away_wins': result[1], 'matches_drawn': result[3], 'total_matches': result[2]})
#     return result
#     return pd.Series({'home_wins': result_break(x)[0], 'away_wins': result_break(x)[1], 'matches_drawn': result_break(x)[3], 'total_matches': result_break(x)[2]})

In [32]:
series_df = series.merge(series.apply(lambda x: result_break(x), axis=1), left_index=True, right_index=True)
series_df.head()

,datadaysID,link,teams_text,years,winner,result,Home,Away,start,end,home_wins,away_wins,matches_drawn,total_matches
0,685654,/ci/engine/series/60260.html,England in Australia Test Series,1876/77,drawn,1-1 (2),Australia,England,1876,1877,1,1,0,2
1,686294,/ci/engine/series/60261.html,England in Australia Test Match,1878/79,Australia,1-0 (1),Australia,England,1878,1879,1,0,0,1
2,686907,/ci/engine/series/60262.html,Australia in England Test Match,1880,England,1-0 (1),England,Australia,1880,1880,1,0,0,1
3,687459,/ci/engine/series/60263.html,England in Australia Test Series,1881/82,Australia,2-0 (4),Australia,England,1881,1882,2,0,2,4
4,687627,/ci/engine/series/60264.html,Australia in England Test Match,1882,Australia,1-0 (1),England,Australia,1882,1882,0,1,0,1


## Retain relevant features

In [33]:
series_df_cleaned = series_df[['Home', 'Away', 'start', 'end', 'winner', 'home_wins', 'away_wins', 'matches_drawn', 'total_matches']]
series_df_cleaned.head()

,Home,Away,start,end,winner,home_wins,away_wins,matches_drawn,total_matches
0,Australia,England,1876,1877,drawn,1,1,0,2
1,Australia,England,1878,1879,Australia,1,0,0,1
2,England,Australia,1880,1880,England,1,0,0,1
3,Australia,England,1881,1882,Australia,2,0,2,4
4,England,Australia,1882,1882,Australia,0,1,0,1


### Export cleaned and uncleaned data to csv files

In [34]:
# Store the data in csv file

#uncleaned df
series_df_uncleaned = pd.DataFrame(records, columns=['datadaysID', 'link', 'teams_text', 'years', 'winner', 'result'])
series_df_uncleaned.to_csv('test_series_uncleaned')
series_df.to_csv('test_series_mix')
series_df_cleaned.to_csv('test_series')

## Exploratory Data Analysis on cleaned dataset

In [35]:
series_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 737
Data columns (total 9 columns):
Home             730 non-null object
Away             730 non-null object
start            730 non-null int64
end              730 non-null int64
winner           730 non-null object
home_wins        730 non-null int64
away_wins        730 non-null int64
matches_drawn    730 non-null int64
total_matches    730 non-null int64
dtypes: int64(6), object(3)
memory usage: 77.0+ KB


In [36]:
series_df_cleaned.describe()

,start,end,home_wins,away_wins,matches_drawn,total_matches
count,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000
mean,1984.858904,1983.909589,1.310959,0.815068,1.041096,3.167123
std,30.989765,32.791272,1.173795,0.968668,1.117738,1.281723
min,1876.000000,1877.000000,0.000000,0.000000,0.000000,1.000000
25%,1972.000000,1970.000000,0.000000,0.000000,0.000000,2.000000
50%,1996.000000,1995.000000,1.000000,1.000000,1.000000,3.000000
75%,2007.000000,2007.000000,2.000000,1.000000,2.000000,4.000000
max,2018.000000,2019.000000,5.000000,5.000000,5.000000,7.000000


### Few points to note
- First test took place in 1876 and ended in 1877
- home wins are most common in a series followed by draws. (1.31 and 1.04)
- An average of 3 matches were played in a series
- There's lot of deviation in team wins and draws
- Maximum no of matches played in a series are 7 while minimum is 1
- Maximum wins for a home or away is 5

In [37]:
print("Teams who haven't won a home game yet : {}\n".format(set(test_playing_nations) - set(series_df_cleaned['Home'].unique())))

print("Teams who haven't won an away game yet : {}\n".format(set(test_playing_nations) - set(series_df_cleaned['Away'].unique())))

Teams who haven't won a home game yet : {'Afghanistan'}

Teams who haven't won an away game yet : {'Ireland'}



### Countrywise data

In [38]:
series_df_pivot = series_df_cleaned.pivot_table(index=['Home'], columns=['Away'], values=['home_wins', 'away_wins', 'total_matches'], aggfunc=np.mean)

In [39]:
import seaborn as sns
import matplotlib.pyplot as plt
fig, ax = plt.subplots(3, 1, figsize=(10, 18))

sns.heatmap(series_df_pivot['total_matches'], cmap='YlOrBr', ax=ax[0])
ax[0].set_title('Avg no of matches played between 2 teams (per season)')

sns.heatmap(series_df_pivot['home_wins'], cmap='YlOrBr', ax=ax[1])
ax[1].set_title('Avg Wins by home team (per season)')

sns.heatmap(series_df_pivot['away_wins'], cmap='YlOrBr', ax=ax[2])
ax[2].set_title('Avg Wins by Away team (per season)')

plt.tight_layout()

In [40]:
# lets check for total matches played, wins and draws by a team across years
series_home = series_df_cleaned.groupby(['Home', 'start'])['home_wins', 'matches_drawn', 'total_matches'].sum()
series_away = series_df_cleaned.groupby(['Away', 'start'])['away_wins', 'matches_drawn', 'total_matches'].sum()

In [116]:
df3 = pd.concat([series_home, series_away], axis=1).sort_index()
df3 = df3.fillna(0)
df3 = df3.groupby(level=0, axis=1).apply(lambda x: x.apply(sum, axis=1))
df3['total_wins'] = df3['away_wins'] + df3['home_wins']
df3['percent_wins'] = (df3['total_wins']/df3['total_matches'])*100
df3 = df3.drop(columns=['away_wins', 'home_wins'])
df3.head()

matches_drawn  total_matches  total_wins  percent_wins
Afghanistan 2018            0.0            1.0         0.0           0.0
Australia   1876            0.0            2.0         1.0          50.0
            1878            0.0            1.0         1.0         100.0
            1880            0.0            1.0         0.0           0.0
            1881            2.0            4.0         2.0          50.0

In [117]:
df3.index = df3.index.set_levels([df3.index.levels[0], pd.to_datetime(df3.index.levels[1], format='%Y')])
df3.index.names = ['Country', 'Date']
df3.head()

matches_drawn  total_matches  total_wins  percent_wins
Country     Date                                                              
Afghanistan 2018-01-01            0.0            1.0         0.0           0.0
Australia   1876-01-01            0.0            2.0         1.0          50.0
            1878-01-01            0.0            1.0         1.0         100.0
            1880-01-01            0.0            1.0         0.0           0.0
            1881-01-01            2.0            4.0         2.0          50.0

In [123]:
# plt.plot(df3.loc['Australia'].index, df3.loc['Australia']['total_matches'])
# plt.plot(df3.loc['Australia'].index, df3.loc['Australia']['matches_drawn'])
# plt.plot(df3.loc['Australia'].index, df3.loc['Australia']['percent_wins'])
# plt.legend(loc='upper left')
# # plt.figure(figsize=(12, 8))
# plt.show()

# df3.groupby(pd.Grouper(freq='10A', level=-1)).sum()

# pd.to_datetime(df3.index, format='%Y')


# fig, ax = plt.subplots(1, 1, figsize=(12, 8))
# df3.loc['Australia']['percent_wins'].plot(use_index=True, figsize=(12, 8))
# df3.loc['Australia']['total_matches'].plot(use_index=True)


,matches_drawn,total_matches,total_wins,percent_wins
Date,,,,
1876-12-31,0.0,4.0,2.0,100.000000
1886-12-31,8.0,48.0,20.0,525.000000
1896-12-31,4.0,52.0,24.0,763.333333
1906-12-31,20.0,80.0,30.0,600.000000
1916-12-31,16.0,84.0,34.0,615.000000
1926-12-31,24.0,66.0,21.0,350.000000
1936-12-31,60.0,184.0,62.0,976.562882
1946-12-31,30.0,50.0,10.0,285.000000
1956-12-31,114.0,308.0,97.0,1703.643579


### 3.1 Univariate Analysis

In [43]:
series_df_cleaned.head(2)

,Home,Away,start,end,winner,home_wins,away_wins,matches_drawn,total_matches
0,Australia,England,1876,1877,drawn,1,1,0,2
1,Australia,England,1878,1879,Australia,1,0,0,1
